# common_to_all_curves
To peform sanity checks on the input data and the algorithm parameter struct. Massage the data (i.e. drop outliers, zscore data, etc)

**USAGE**:
[OUT] = COMMON_TO_ALL_CURVES(CURVE_TYPE, GET_INFO, VARARGIN)

**INPUTS**:
- curve_type: Family of curves, string, e.g. 'free_rmpw'
- get_info: Cues for specific information / computation, string, e.g. 'initial_sampling'
- varargin: Has arguments depending on the computation

**OUTPUTS**:
- out: the output of all computations

In [ ]:
import numpy as np
def common_to_all_curves(curve_type, get_info, *varargin):
    # Checks if input arguments are passed in
    if len(varargin) == 0:
        raise ValueError('No input arguments!')
        
    if get_info is 'initial_sampling':
        if len(varargin) < 2:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))
        nParticles = varargin[0]
        if nParticles <= 0:
            raise ValueError('Number of particles will need to > 0!')
        resolution = varargin[1]
        if resolution <= 0:
            raise ValueError('Resolution will need to > 0!')
        
        bounds = family_of_curves(curve_type, 'get_bounds')
        nParams = family_of_curves(curve_type, 'get_nParams')
        out = np.full((nParticles, nParams), np.nan)

        # Uniform sampling each curve parameter bounded by its respective bounds
        for i in range(nParams):
            out[:, i] = np.random.uniform(low=bounds[i, 1], high=bounds[i, 2], size=(nParticles, 1))

        out = round_to(out, resolution)
        if np.any(np.isnan(out)):
            raise ValueError('NaNs in initial sampling output matrix!')
        
    elif get_info is 'check_if_exceed_bounds':
        if len(varargin) < 1:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))
            
        out = varargin[0]
        nParams = family_of_curves(curve_type, 'get_nParams')
        if (not out) or (np.shape(out)[1] != nParams):
            raise ValueError('Not a valid input matrix!')

        bounds = family_of_curves(curve_type, 'get_bounds')
        nParams = family_of_curves(curve_type, 'get_nParams')

        # If a curve parameter is found to exceeding bounds then it is set to the bounds
        # For instance if a vertical parameter is -1.02 then it is set to -1 since -1 is the lower bound for vertical parameters
        for i in range(nParams):
            out[:, i] = max(out[:, i], bounds[i, 1])
            out[:, i] = min(out[:, i], bounds[i, 2])
    
    elif get_info is 'curve_volumes':
        if len(varargin) < 1:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))

        resolution = varargin[0]
        if resolution <= 0:
             raise ValueError('Resolution will need to > 0!')

        bounds = family_of_curves(curve_type, 'get_bounds')
        nParams = family_of_curves(curve_type, 'get_nParams')
        total_vol = 1

        # Lebesgue measure http://en.wikipedia.org/wiki/Lebesgue_measure
        for i in range(nParams):
            total_vol = total_vol * len(np.arange(bounds[i, 1], bounds[i, 2], 1/10^resolution))
        out[0] = total_vol
        
    elif get_info is 'flip_vertical_params':
        if len(varargin) < 1:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))
            
        input_params = varargin[0]
        nParams = family_of_curves(curve_type, 'get_nParams')
        if (not input_params) or (np.shape(input_params)[1] != nParams):
            raise ValueError('Not a valid input matrix!')
            
        out = input_params
        vertical_params = family_of_curves(curve_type, 'get_vertical_params_only')

        # Flipping vertical parameters of the curve. If a y1 = -0.4, flipping it will result in 0.4
        for i in range(len(vertical_params)):
            out[:, vertical_params[i]] = np.multiply(input_params[:, vertical_params[i]], -1)

    elif get_info is 'sort_horizontal_params':
        if len(varargin) < 1:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))
            
        input_params = varargin[0]
        nParams = family_of_curves(curve_type, 'get_nParams')
        if (not input_params) or (np.shape(input_params)[1] != nParams):
            raise ValueError('Not a valid input matrix!')
            
        out = input_params
        horizontal_params = family_of_curves(curve_type, 'get_horizontal_params_only')
        if len(horizontal_params) != 2:
            raise ValueError('Incorrect horizontal parameters count for {} family of curves'.format(curve_type))

        # This piece of code ensures that x1 <= x2 especially for the horz_indpnt family of curves
        idx = input_params[:, horizontal_params[0]] > input_params[:, horizontal_params[1]]
        out[idx, horizontal_params[0]] = input_params[idx, horizontal_params[1]]
        out[idx, horizontal_params[1]] = input_params[idx, horizontal_params[0]]

        if not np.all(out[:, horizontal_params[0]] <= out[:, horizontal_params[1]]):
            raise ValueError('Horizontal parameter 1 is NOT <= Horizontal parameter 2 in {} family of curves'.format(curve_type))

    case 'draw_bcm_curve'
        if len(varargin) < 2:
            raise ValueError('Missing input parameters in {} computation!'.format(get_info))
        input_params = varargin[0]
        resolution = varargin[1]
        if resolution <= 0:
            raise ValueError('Resolution will need to > 0!')
                
        # This draws a BCM curve for you. If you passed in the 'input_params' as 'con' then it randomly draws a theory consistent curve; 'inc' - theory inconsistent curve
        if input_params is 'con' or input_params is 'inc':
            input_params = common_to_all_curves(curve_type, 'auto_generate', input_params, resolution)
            
        nParams = family_of_curves(curve_type, 'get_nParams')
        if (not input_params) or (np.shape(input_params)[1] != nParams):
            raise ValueError('Not a valid input matrix!')

        # If instead you passed in [y1, x1, x2, y2, y3 and y4] into 'input_params' then it draws a curve directly rather then randomly generating one for you
        out = family_of_curves(curve_type, 'get_curve_xy_vals', input_params)

        figure(); set(gcf, 'Position', [800, 900, 600, 600]);
        plot(out.xval, out.yval, 'LineWidth', 3); grid on;
        xlabel('Activation', 'FontSize', 18); ylabel('Change in Memory Strength', 'FontSize', 18);
        ylim([-1.2, 1.2]); title(out.title_string);

